In [1]:
import numpy as np
import Cluster_Ensembles as CE
from functools import reduce

In [2]:
# require(data.table)
# require(bit64)
# require(dbscan)
# require(doParallel)
# require(rBayesianOptimization)
# path='../input/train_1/'
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from trackml.randomize import shuffle_hits

from sklearn.preprocessing import StandardScaler
import hdbscan as _hdbscan
from scipy import stats
from tqdm import tqdm

import time

from sklearn.cluster.dbscan_ import dbscan
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KDTree
import hdbscan
from bayes_opt import BayesianOptimization

# https://www.ellicium.com/python-multiprocessing-pool-process/
# http://sebastianraschka.com/Articles/2014_multiprocessing.html
from multiprocessing import Pool

In [3]:
import os
import time
import hdbscan as _hdbscan
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [4]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [5]:
def preprocess(hits):        
    x = hits.x.values
    y = hits.y.values
    z = hits.z.values

    r = np.sqrt(x**2 + y**2 + z**2)
    hits['x2'] = x/r
    hits['y2'] = y/r

    r = np.sqrt(x**2 + y**2)
    hits['z2'] = z/r

    ss = StandardScaler()
    X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
#     for i, rz_scale in enumerate(self.rz_scales):
#         X[:,i] = X[:,i] * rz_scale

    return X

In [6]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#------------------------------------------------------

def make_counts(labels):
    
    
    _,reverse,count = np.unique(labels,return_counts=True,return_inverse=True)
    counts = count[reverse]
    counts[labels==0]=0
    
    return counts

def one_loop(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r, a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da 
    zr = z/r
    
    X = StandardScaler().fit_transform(np.column_stack([aa, aa/zr, zr, 1/zr, aa/zr + 1/zr]))
    _,l = dbscan(X, eps=0.0035, min_samples=1,)

    return l

def one_loop1(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def one_loop2(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def do_dbscan_predict(df):
    
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    r  = np.sqrt(x**2+y**2)
    d  = np.sqrt(x**2+y**2+z**2)
    a  = np.arctan2(y,x)

    x2 = df['x']/d
    y2 = df['y']/d
    z2 = df['z']/r

    r2 = np.sqrt(x2**2 + y2**2)
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    phi2 = np.arctan2(y2, x2)
    phi2_deg = np.degrees(np.arctan2(y2, x2))
    
    for angle in range(-180,180,1):
    
        df1 = df.loc[(df.phi_deg>(angle-1.0)) & (df.phi_deg<(angle+1.0))]
    
        x  = df1.x.values
        y  = df1.y.values
        z  = df1.z.values
        r  = np.sqrt(x**2+y**2)
        d  = np.sqrt(x**2+y**2+z**2)
        a  = np.arctan2(y,x)

        x2 = df1['x']/d
        y2 = df1['y']/d
        z2 = df1['z']/r
    
        r2 = np.sqrt(x2**2 + y2**2)
        theta= np.arctan2(r, z)
        theta1 = np.arctan2(r2, z2)

    
        tan_dip = phi/theta
        tan_dip1 = phi/z2
        z2_1 = 1/z2
        z2_2 = phi/z2 + 1/z2

        dip_angle = np.arctan2(z2, (np.sqrt(x2**2 +y2**2)) * np.arccos(x2/np.sqrt(x2**2 + y2**2)))
        dip_angle1 = np.arctan2(z, (np.sqrt(x**2 +y**2)) * np.arccos(x2/np.sqrt(x**2 + y**2)))
        scores = []

        a_start,a_step,a_num = 0.00100,0.0000095,150
    
        params  = [(i,m, x,y,z,d,r,r2,z2, a, a_start,a_step) for i in range(a_num) for m in [-1,1]]

        if 1: 
            pool = Pool(processes=1)
            ls   = pool.map( one_loop1, params )


        if 0:
            ls = [ one_loop(param) for param in  params ]


        ##------------------------------------------------

        num_hits=len(df)
        labels = np.zeros(num_hits,np.int32)
        counts = np.zeros(num_hits,np.int32)
        for l in ls:
            c = make_counts(l)
            idx = np.where((c-counts>0) & (c<20))[0]
            labels[idx] = l[idx] + labels.max()
            counts = make_counts(labels)
       

#     cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
#                          metric='braycurtis',cluster_selection_method='leaf',algorithm='best', 
#                          leaf_size=50)
    
#     X = preprocess(df)
#     l1 = pd.Series(labels)
#     labels = np.unique(l1)
   
# #   print(X.shape)
# #   print(len(labels_org))
# #   print(len(labels_org[labels_org ==0]))
# #   print(len(labels_org[labels_org ==-1]))
    
#     n_labels = 0
#     while n_labels < len(labels):
#         n_labels = len(labels)
#         max_len = np.max(l1)
#         s = list(l1[l1 == 0].keys())
#         X = X[s]
#         print(X.shape)
#         if X.shape[0] <= 1:
#             break
#         l = cl.fit_predict(X)+max_len
# #         print(len(l))
#         l1[l1 == 0] = l
#         labels = np.unique(l1)
    
    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

def extend(submission,hits):
    df = submission.merge(hits,  on=['hit_id'], how='left')
#     df = submission.append(hits)
#     print(df.head())
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in range(-180,180,1):

        print ('\r %f'%angle, end='',flush=True)
        #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
        df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

        min_num_neighbours = len(df1)
        if min_num_neighbours<4: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        tree = KDTree(np.column_stack([a,r]), metric='euclidean')

        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0) 

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1) 


            ## extend start point
            ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)
            direction = np.arctan2(r0-r[ns],a0-a[ns])
            ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

            ## extend end point
            ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns]-r1,a[ns]-a1)
            ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
    #print ('\r')
#     df = df[['particle_id', 'weight', 'event_id', 'hit_id', 'track_id']]
    df = df[['event_id', 'hit_id', 'track_id']]
    return df

In [7]:
import hdbscan
seed = 123
np.random.seed(seed)

def shift(l, n):
    return l[n:] + l[:n]

def trackML31(df, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14, w15, w16, w17, w18, Niter, z_shift):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    
    dz = 0
    z = z + dz
    
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    x2 = x/r
    y2 = y/r
    
    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    z3 = np.log1p(abs(z/r))*np.sign(z)
    
    x1 = x/rt
    y1 = y/rt
    
    y3 = np.log1p(abs(y/r))*np.sign(y)
    
    theta = np.arctan2(rt, z)
    theta_deg = np.degrees(np.arctan2(rt, z))
    tt = np.tan(theta_deg)
    
    z4 = np.sqrt(abs(z/rt))
    
    x4 = np.sqrt(abs(x/r))
    y4 = np.sqrt(abs(y/r))
    
    mm = 1
    ls = []
    
#     print(Niter)
#     Niter = Niter.astype(np.int32)
#     print(Niter)
#     print(w1, w2, w3, Niter)
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
#         a1 = a0+mm*(rt+0.000045*rt**2)/1000*(ii/2)/180*np.pi
#         a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
        a1 = a0+mm*(rt+ 0.0000145*rt**2)/1000*(ii/2)/180*np.pi
    
        da1 = mm*(1 + (2 * 0.0000145 * rt))/1000*(ii/2)/180*np.pi
        
        ia1 = a0*rt + mm*(((rt**2)/2) + (0.0000145*rt**3)/3)/1000*(ii/2)/180*np.pi
        
        sa1 = np.sin(da1)
        ca1 = np.cos(da1)
        
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        raa = x*caa + y*saa
        
        t1 = theta+mm*(rt+ 0.8435*rt**2)/1000*(ii/2)/180*np.pi
        ctt = np.cos(t1)
        stt = np.sin(t1)
        ttt = np.tan(t1)
        
#         dt1 = mm * (1 + (2 * 0.8435 * rt))/1000*(ii/2)/180*np.pi
        
        r0Inv = 2. * np.cos(a1 - t1) / r
        r0Inv1 = 2. * np.cos(a1 - t1) / rt
        
        t0 = a0 - theta
        b1 = t0+mm*(rt+ 0.0000005*rt**2)/1000*(ii/2)/180*np.pi
        r0Inv2 = 2. * np.cos(b1) / r
        
        deps = 0.0000025
#         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, z3, ctt, stt]))
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, y/r, z3, y1, y3, 
                                                            ctt, stt, z4, x4, y4, raa, r0Inv, da1]))
       
        
#         cx = [w1,w1,w2,w3, w4, w5, w6, w7, w7]
        cx = [w1,w1,w2,w3, w4, w5, w6, w7, w8, w9, w10, w10, w11, w12, w13, w14, w15, w16]
#         print(X.shape, len(cx))
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        deps = 0.0000025
#         cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,cluster_selection_method='leaf',metric='euclidean', leaf_size=50)
#         l = cl.fit_predict(X)+1
        
        l= DBSCAN(eps=0.004,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#         l= DBSCAN(eps=w7,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
    
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    lss = []
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
    
    lss.append(labels)
    
    for i in range(Niter):
        labels1 = np.zeros(num_hits,np.int32)
        counts1 = np.zeros(num_hits,np.int32)
        ls1 = ls.copy()
        ls1 = shift(ls1, 1)
        np.random.shuffle(ls1)
        for l in ls1:
            c = make_counts(l)
            idx = np.where((c-counts>0) & (c<20))[0]
            labels1[idx] = l[idx] + labels1.max()
            counts1 = make_counts(labels1)
        l1 = labels1.copy()
        lss.append(l1)


    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    
    for l in lss:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
    

#     sub = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
#         data=np.column_stack(([int(0),]*len(df), df.hit_id.values, labels))
#     )
#     sub['track_count'] = sub.groupby('track_id')['track_id'].transform('count')

    
#     sub1 = sub[sub.track_id <  0]
#     sub2 = sub[sub.track_id >= 0]
#     L_neg = sub1.track_id.values
#     L_pos = sub2.track_id.values
#     a = 1
#     for l in L_neg:
#         for l1 in range(a, np.iinfo(np.int32).max):
#             if l1 in L_pos:
#                 continue
#             sub.loc[sub.track_id == l, 'track_id'] = l1
#             a = l1 +1
#             break
    
#     L = list(sub.track_id.values)
#     labels = np.zeros(num_hits,np.int32)
#     for ii in range(num_hits):
#         labels[ii] = L[ii]
#     print(np.any(labels < 0))
    return labels

In [8]:
def Fun4BO2(w18):
    
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = 0.1470
    w5 = 0.01201
    w6 = 0.0003864
    w7 = 0.0205
    w8 = 0.0049
    w9 = 0.00121
    w10 = 1.4930496676654575e-05
    w11 = 0.0318
    w12 = 0.000435
    w13 = 0.00038
    w14 = 0.00072
    w15 = 0.000265
    w16 = 0.0031
    w17 = 0.00021
#     Niter=179
#     Niter=246
    Niter=247
    

    print(w18)
    z_shift = 0
#     dz = z_shift
#     hits.z = hits.z.values + dz
    
#     Niter = Niter.astype(np.int32)
    track_id = trackML31(hits, w1,w2,w3,w4,w5,w6,w7,w8,w9,w10, w11, w12, w13, w14, w15, w16, w17, w18, Niter, 
                         z_shift)
    
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(8):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return sc

In [31]:
# def run_dbscan():

data_dir = '../data/train'

# event_ids = [
# # #             '000001030',##
#             '000001025','000001026',
# '000001027','000001028','000001029',
# ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
#     w1 = [0.9, 1.2]
#     w2 = [0.3, 0.7]
#     w3 = [0.1, 0.4]
#     Niter = [140, 190]
    
    
# #     bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
# #     bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
#     w1 = 1.1932215111905984
#     w2 = 0.39740553885387364
#     w3 = 0.3512647720585538
#     w4 = [0.1, 0.2] # 0.1470 -> 0.55690
#     w4 = 0.1470
#     w5 = [0.001, 1.2] # 0.7781 -> 0.55646, 0.7235 + N = 247 => 0.56025
    
#     Niter = 179
#     Niter = 247
#     w5 = 0.01
# #     for w6 in [0.012, 0.01201, 0.01202, 0.01203, 0.01204, 0.01205, 0.01206, 0.01207, 0.01208, 0.01209, 0.0121]:
#     EPS = 1e-12
#     w6 = [0.001, 1.2]
#     w6 = 0.0205
#     w8 = [0.00008, 0.0005]
#     w13 = 0.00038
#     Fun4BO2(0)
#     for z_shift in np.arange(36, 45, 1):
#         print(z_shift)
#         Fun4BO2(z_shift)
#     for w18 in [5.0, 4.0, 3.0]:
#         print(w18)
    Fun4BO2(0)
#     for Niter in np.arange(240, 250, 1):
#         Fun4BO2(Niter)
#     Niter = [240, 480]
#     w16 = [0.000001, 0.00001]
#     bo = BayesianOptimization(Fun4BO2,pbounds = {'w16':w16})
#     bo.maximize(init_points = 20, n_iter = 5, acq = "ucb", kappa = 2.576)

# x/y: 7 | 06m30s |    0.55302 |    0.0100 | 
# x/y: 0.001: 0.55949
# x/y: 0.0001: 0.55949
# x/y: 0.002: 0.55959
# x/y: 0.003: 0.55915
# x/y: 0.0025: 0.55925
# x/y: 0.0015: 0.55953
# x/r: 0.0015: 0.56186
# x/r: 0.002: 0.56334
# x/r: 0.0025: 0.563989
# x/r: 0.003: 0.56447
# x/r: 0.01: 0.569822
# x/r: 0.015: 0.56940
# x/r: 0.012: 0.5719
# x/r: 0.01201: 0.57192
# 1.4499999999999993e-05 * rt**2: 0.5720702851970194
# 0.0000145
# z3: 10 | 07m12s |    0.57208 |    0.0205 | 
# count: 19: 0.572567, 17: 0.57263
# ctt, stt after change:  2 | 07m56s |    0.57345 |    0.0001 | (0.00010567777727496665)
# x4: 25 | 09m42s |    0.57359 |    0.0002 | (0.000206214286412982)
# x4: 0.000435 (0.5737387485278771) (x4 = np.sqrt(abs(x/r)))
# w13: 00038 (ctt,stt): 0.5737528800479372
# ensemble of 10: 0.5772859116242378
# ensemble of Niter=247 (random shuffle+ shift): 0.5787580886742594
# ensemble of Niter=247 (random shuffle+ shift + epsilon 0.004): 0.5865991424251623
# Niter: 245 (0.5871422948224762) 355 (0.5869811038892895) 415 (0.5870650622208972) 460 (0.5873663083421805)
# Niter: 246 (0.587203515195237)
# testing r0Inv (3 | 25m18s |    0.58483 |    0.0012 | )
# testing r0Inv (5.0e-05, 0.5876165835032465) (0.00038)
# testing w15: 0.000265 - 0.5878985119131975
# testing w15: 5.5e-05: 0.587602145623185
# testing w15 = 0.000265: 0.5878840726381361 (after reset)
# z_shift: -0.0055: 0.5884188919516644
# z_shift: 0.0015 (0.5881407134817294)
# z_shift: 40: 0.5866796958355055 (strange)
# fresh: 0.5774021620422332
# X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, y/r, z3, y1, y3, 
#                                                             ctt, stt, z4, x4, y4, raa, r0Inv]))
# cx = [w1,w1,w2,w3, w4, w5, w6, w7, w8, w9, w10, w10, w11, w12, w13, w14, w15]
# da1: w16: 0.0031: 0.5860334935502527
# testing ia1 (0.001: 0.5841576358060114, 0.0008:0.5850962676812601, 0.00021: 0.5861659551095669)
# testing dt1
# org: 0.5861659551095669 (same for 0.01, 0.001)
# org: 0.5903235438697783
# without ia1: 0.5903525691536519
# 0.557005489415385, 0.5604565570541568, 0.572282204131876, 0.5697993366784551, 0.5914246425600556 
# with ia1: 0.5570087874489997, 0.5605272680265565, 0.5722191378409494, 0.5697114879199746, 0.5912763773013219
# without ia1 and 19: 0.556987423236126, 0.5604565570541568 (less)

0
 179.0000000[ 0] score : 0.54665037
 179.0000000[ 1] score : 0.55447094
 179.0000000[ 2] score : 0.55773667
 179.0000000[ 3] score : 0.55872481
 179.0000000[ 4] score : 0.55899811
 179.0000000[ 5] score : 0.55956275
 179.0000000[ 6] score : 0.55962888
 179.0000000[ 7] score : 0.56012687
--------------------------------------
0.556987423236126
0
 179.0000000[ 0] score : 0.55054109
 179.0000000[ 1] score : 0.55927198
 179.0000000[ 2] score : 0.56088157
 179.0000000[ 3] score : 0.56202533
 179.0000000[ 4] score : 0.56266099
 179.0000000[ 5] score : 0.56264426
 179.0000000[ 6] score : 0.56274872
 179.0000000[ 7] score : 0.56287851
--------------------------------------
0.5604565570541568
0


KeyboardInterrupt: 

In [2]:
cluster_runs = np.random.randint(0, 50, (50, 15000))

In [3]:
cluster_runs.shape

(50, 15000)

In [3]:
consensus_clustering_labels = CE.cluster_ensembles(cluster_runs, verbose = True, N_clusters_max = 50)


INFO: Cluster_Ensembles: cluster_ensembles: due to a rather large number of cells in your data-set, using only 'HyperGraph Partitioning Algorithm' (HGPA) and 'Meta-CLustering Algorithm' (MCLA) as ensemble consensus functions.


*****
INFO: Cluster_Ensembles: HGPA: consensus clustering using HGPA.

#
INFO: Cluster_Ensembles: wgraph: writing wgraph_HGPA.
INFO: Cluster_Ensembles: wgraph: 15000 vertices and 2500 non-zero hyper-edges.
#

#
INFO: Cluster_Ensembles: sgraph: calling shmetis for hypergraph partitioning.
INFO: Cluster_Ensembles: sgraph: (hyper)-graph partitioning completed; loading wgraph_HGPA.part.50
#

INFO: Cluster_Ensembles: cluster_ensembles: HGPA at 0.02434736018512536.
*****

*****
INFO: Cluster_Ensembles: MCLA: consensus clustering using MCLA.
INFO: Cluster_Ensembles: MCLA: preparing graph for meta-clustering.
INFO: Cluster_Ensembles: MCLA: done filling hypergraph adjacency matrix. Starting computation of Jaccard similarity matrix.
INFO: Cluster_Ensembles: MCLA: startin

In [4]:
cluster_runs.shape

(50, 15000)

In [5]:
consensus_clustering_labels.shape

(15000,)

In [8]:
w1 = 1.1932215111905984
w2 = 0.39740553885387364
w3 = 0.3512647720585538
w4 = 0.1470
w5 = 0.01201
w6 = 0.0003864
w7 = 0.0205
w8 = 0.0049
w9 = 0.00121
w10 = 1.4930496676654575e-05
w11 = 0.0318
w12 = 0.000435
w13 = 0.00038
w14 = 0.00072
w15 = 0.000265
w16 = 0.0031
w17 = 0.00021
w18 = 0
Niter=247
z_shift = 0

In [9]:
def run_make_submission():

    data_dir = '../data/test'


    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = 0.1470
    w5 = 0.01201
    w6 = 0.0003864
    w7 = 0.0205
    w8 = 0.0049
    w9 = 0.00121
    w10 = 1.4930496676654575e-05
    w11 = 0.0318
    w12 = 0.000435
    w13 = 0.00038
    w14 = 0.00072
    w15 = 0.000265
    w16 = 0.0031
    w17 = 0.00021
    w18 = 0
    Niter=247
    z_shift = 0
    tic = t = time.time()
    event_ids = [ '%09d'%i for i in range(0,125) ]  #(0,125)

    if 1:
        submissions = []
        for i,event_id in enumerate(event_ids):
            hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
            cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)

#             track_id = trackML2(hits, w1, w2, w3, Niter)
            track_id = trackML31(hits, w1,w2,w3,w4,w5,w6,w7,w8,w9,w10, w11, w12, w13, w14, w15, w16, w17, w18, 
                                 Niter, z_shift)
            #track_id = back_fit(track_id,hits)

            toc =  time.time()
            print('\revent_id : %s  , %0.0f min'%(event_id, (toc-tic)/60))

            # Prepare submission for an event
            submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
                data=np.column_stack(([event_id,]*len(hits), hits.hit_id.values, track_id))
            ).astype(int)
            submissions.append(submission)
            
            for i in range(8):
                submission = extend(submission,hits)
            
            
            submission.to_csv('../cache/sub3/%s.csv.gz'%event_id,
                                index=False, compression='gzip')

            #------------------------------------------------------
    if 1:

        event_ids = [ '%09d'%i for i in range(0,125) ]  #(0,125)
        submissions = []
        for i,event_id in enumerate(event_ids):
            submission  = pd.read_csv('../cache/sub3/%s.csv.gz'%event_id, compression='gzip')
            submissions.append(submission)

        
        # Create submission file
        submission = pd.concat(submissions, axis=0)
        submission.to_csv('../submissions/sub3/submission-0028.csv.gz',
                            index=False, compression='gzip')
        print(len(submission))

In [10]:
run_make_submission()

event_id : 000000000  , 19 min
event_id : 000000001  , 42 min
event_id : 000000002  , 61 min
event_id : 000000003  , 80 min
event_id : 000000004  , 101 min
event_id : 000000005  , 121 min
event_id : 000000006  , 141 min
event_id : 000000007  , 162 min
event_id : 000000008  , 184 min
event_id : 000000009  , 204 min
event_id : 000000010  , 225 min
event_id : 000000011  , 245 min
event_id : 000000012  , 264 min
event_id : 000000013  , 287 min
event_id : 000000014  , 306 min
event_id : 000000015  , 322 min
event_id : 000000016  , 339 min
event_id : 000000017  , 362 min
event_id : 000000018  , 381 min
event_id : 000000019  , 399 min
event_id : 000000020  , 415 min
event_id : 000000021  , 435 min
event_id : 000000022  , 459 min
event_id : 000000023  , 483 min
event_id : 000000024  , 506 min
event_id : 000000025  , 528 min
event_id : 000000026  , 547 min
event_id : 000000027  , 566 min
event_id : 000000028  , 586 min
event_id : 000000029  , 603 min
event_id : 000000030  , 624 min
event_id : 0